In [8]:
import requests
from bs4 import BeautifulSoup
import re
#The basics
import numpy as np
import pandas as pd
import pickle

#Get them web sites
import requests

#Make sure slenium works
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

#Start the google driver
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
#For inserting articles into Mongodb
from pymongo import MongoClient

In [9]:
actpost_first_articles = pickle.load(open("/Users/NickThomas/Project5/pickle_files/actpost_first_articles.pkl", "rb" ))
actpost_more_articles = pickle.load(open("/Users/NickThomas/Project5/pickle_files/actpost_more_articles.pkl", "rb" ))

In [21]:
actpost_more_articles

['https://www.activistpost.com/2018/04/1986-cia-document-analyzes-possibilities-of-regime-change-in-syria.html',
 'https://www.activistpost.com/2018/04/bill-gates-warns-of-coming-apocalyptic-disease-pledges-12-million-to-universal-vaccine.html',
 'https://www.activistpost.com/2018/05/israel-complains-syria-is-defending-itself.html',
 'https://www.activistpost.com/2018/05/us-history-attempting-overthrow-iran-riddled-with-conspiracy-to-stage-conflicts-disbandthecia.html',
 'https://www.activistpost.com/2018/04/in-2011-ohios-dennis-kucinich-spoke-about-harm-from-exposure-to-cell-phones-and-cell-towers-so-where-does-he-stand-now-on-bills-which-allow-small-cell-towers-in-yards.html',
 'https://www.activistpost.com/2018/05/study-of-91000links-nighttime-phone-use-with-mental-health-disorders-decades-of-research-says-cell-phone-and-wifi-radiation-exposure-can-cause-same-disorders-24-7.html',
 'https://www.activistpost.com/2018/04/town-bypasses-constitution-us-citizens-given-60-days-to-turn-in-

In [10]:
def get_article(url):
#Sends request for url
    html = requests.get(url).text
    return html

def parse_article(html):
#BeautifulSoups the article
    soup = BeautifulSoup(html, 'lxml')
    title = soup.find('h1', class_='entry-title').text
    body = soup.find('div', class_='entry-content clearfix').text

    article = {
        'title': title,
        'body': body,
        'source': 'Activist Post',
        'num_source': 15
    }

    return article

def get_parsed_article_from_link(url):
#Runs the previous two functions on the url
    return parse_article(get_article(url))

In [11]:
#phase 1: uses request to try to BeautifulSoup links
actpost_list_o_articles = []
actpost_problem_articles = []
for text in actpost_first_articles:
    #print(text)
    try:
        art = get_parsed_article_from_link(text.encode())
        actpost_list_o_articles.append(art)

    except:
        #print("Problem processing url " + text)
        problem = text
        actpost_problem_articles.append(problem)
    time.sleep(2)

In [12]:
#phase 2: uses selenium to go through links
driver = webdriver.Chrome(chromedriver)
for x in actpost_problem_articles:
    try:
        driver.get(x)
        time.sleep(3)
        soupy = BeautifulSoup(driver.page_source, 'lxml')
        title = soupy.find('h1', class_='article-header__title').text
        body = soupy.find('div', class_='entry-content clearfix').text

        articley = {
        'title': title,
        'body': body,
        'source': 'Activist Post',
        'num_source': 15
        }
        actpost_list_o_articles.append(articley)
    except:
        pass

In [13]:
#phase 1: uses request to try to BeautifulSoup links
actpost_more_list_o_articles = []
actpost_more_problem_articles = []
for text in actpost_more_articles:
    #print(text)
    try:
        art = get_parsed_article_from_link(text.encode())
        actpost_more_list_o_articles.append(art)

    except:
        #print("Problem processing url " + text)
        problem = text
        actpost_problem_articles.append(problem)
    time.sleep(2)

In [14]:
#phase 2: uses selenium to go through links
driver = webdriver.Chrome(chromedriver)
for x in actpost_more_problem_articles:
    try:
        driver.get(x)
        time.sleep(3)
        soupy = BeautifulSoup(driver.page_source, 'lxml')
        title = soupy.find('h1', class_='article-header__title').text
        body = soupy.find('div', class_='entry-content clearfix').text

        articley = {
        'title': title,
        'body': body,
        'source': 'Activist Post',
        'num_source': 15
        }
        actpost_more_list_o_articles.append(articley)
    except:
        pass

In [15]:
all_articles = actpost_list_o_articles + actpost_more_list_o_articles

In [16]:
len(all_articles)

10

In [17]:
#starts client in Mongodb
client = MongoClient()
test = client.derp.test

In [18]:
#creates event and loads articles into Mongodb
db = client.events
test = db.test
test.insert_many(all_articles)


In [19]:
test.count()

87